(ch:building_blocks)=
# 신경망 구성 요소

**감사의 글**

아래 내용은 프랑소와 숄레의 
[Deep Learning with Python(2판)](https://github.com/fchollet/deep-learning-with-python-notebooks)의 
소스코드 내용을 참고해서 작성되었습니다.
자료를 공개한 저자에게 진심어린 감사를 전합니다.

**소스코드**

여기서 언급되는 코드를
[(구글 코랩) 신경망 구성 요소](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/notebooks/NB-building_blocks_of_NN.ipynb)에서 
직접 실행할 수 있다.

**주요 내용**

아래 요소들을 직관적으로 살펴본다.

- 텐서(tensor)
- 텐서 연산
- 경사 하강법
- 역전파

(sec:nn-mnist)=
## 신경망 모델 활용법

MNIST 손글씨 데이터셋을 대상으로 분류 신경망 모델을 훈련시키고 활용하는 방법을
간단하게 소개한다.

### 훈련셋 준비

여기서는 케라스가 제공하는 MNIST 데이터셋 불러오는 것으로 훈련셋을 준비한다.

```python
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
```

- 손글씨 숫자 인식 용도 데이터셋. 28x28 픽셀 크기의 사진 70,000개의 샘플로 구성
    라벨: 0부터 9까지 10개의 클래스 중 하나
- 훈련셋: 샘플 60,000개 (모델 훈련용)
    - `train_images`
    - `train_labels`
- 테스트셋: 샘플 10,000개 (훈련된 모델 성능 테스트용)
    - `test_images`
    - `test_labels`

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist-7.png?raw=true" style="width:600px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://towardsdatascience.com/exploring-how-neural-networks-work-and-making-them-interactive-ed67adbf9283">Towards data science: Mikkel Duif(2019)</a>&gt;</div></p>

:::{admonition} 샘플, 타깃, 라벨, 예측값, 클래스
:class: info

머신러닝 모델의 훈련에 사용되는 데이터셋과 관련된 기본 용어는 다음과 같다.

- 샘플<font size='2'>sample</font>: 개별 데이터를 가리킴.
- 타깃<font size='2'>target</font>과 라벨<font size='2'>label</font>
    - 타깃: 개별 샘플과 연관된 값이며, 샘플이 주어지면 머신러닝 모델이 맞춰야 하는 값임.
    - 라벨: 분류 과제의 경우 타깃 대신 라벨이라 부름.
- 예측과 예측값: 개별 샘플에 대해 머신러닝 모델이 타깃에 가까운 값을 예측할 수록 좋은 성능의 모델임. 예측값은 모델이 입력 샘플들에 대해 예측한 값.
- 클래스<font size='2'>class</font>: 분류 모델의 에측값으로 사용될 수 있는 라벨(타깃)들의 집합. 범주<font size='2'>category</font>라고도 함. 
    객체지향 프로그래밍 언어의 클래스 개념과 다름에 주의할 것.
:::

### 신경망 모델 지정

다음과 같이 구성된 신경망 모델을 MNIST 분류 모델로 사용한다.

```python
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
    ])
```

위 신경망 모델의 구조에 사용된 요소들은 다음과 같다.

- `Sequential` 클래스
    - 1개 이상의 층을 순차적으로 연결하여 모델 객체를 생성하는 (파이썬) 클래스.
    - 앞으로 다른 방식으로 모델 객체를 생성하는 다양한 클래스를 접할 것임.
- 층<font size='2'>layer</font>
    - 데이터가 입력되면 적절한 방식으로 변환 후 이어지는 층으로 전달함.
    - 여기서는 2개의 `Dense` 층 사용.
- `Dense` 층
    - 입력 샘플의 모든 특성을 이용하여 층의 출력값을 생성함. 
        이런 방식으로 연결된 층들을 **조밀하게 연결된**<font size='2'>densely connected</font> 
        또는 **완전하게 연결된**<font size='2'>fully-connected</font> 층이라고 함.
        `Dense` 층은 항상 조밀하게 다음 층과 연결됨.
    - 첫째 `Dense` 층
        - 512개의 유닛 사용. 784개의 픽셀값으로부터 512개의 값을 생성.
            즉, 한 장의 MNIST 손글씨 숫자 사진에 해당하는 길이가 784인 1차원 어레이가 입력되면
            길이가 512인 1차원 어레이를 생성함.
        - 렐루<font size='2'>Relu</font> 함수: 활성화 함수로 사용됨.
            생성된 512개의 값 중에서 음수는 모두 0으로 처리하는 함수.
    - 둘째 `Dense` 층
        - 10개의 유닛 사용. 입력된 512개의 값으로부터 10개의 값을 생성.
        - 소프트맥스<font size='2'>Softmax</font> 함수가 활성화 함수로 사용됨.
        - 계산된 10개의 값을 이용하여 0부터 9까지 10개의 범주 각각에 속할 확률을 계산함. 모든 확률의 합은 1.
- 유닛<font size='2'>unit</font>
    - 생성된 값을 저장하는 장치.
    - 하나의 유닛에 하나의 값이 저장됨.
- 활성화 함수<font size='2'>activation function</font>
    - 생성되어 유닛에 저장된 값을 이용하여 새로운 개수의 다른 값을 생성하는 함수.
    - 활성화 함수를 통과한 값이 다음 층으로 전달됨.

### 신경망 모델 컴파일

지정된 신경망 모델을 훈련시키기 위해서 옵티마이저, 손실 함수, 성능 평가 지표를 
설정하는 컴파일 과정을 실행해야 한다.

```python
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
```

위 컴파일 과정에 사용된 요소들은 다음과 같다.

- `optimizer`
    - 경사하강법(백워드 패스, 역전파) 업무를 처리하는 옵티마이저 지정.
    - 여기서는 `rmsprop` 옵티마이저 사용.
    - 앞으로 다양한 옵티마이저를 접할 것임.
- `loss`
    - 손실 함수<font size='2'>loss function</font> 지정.
    - 손실 함수: 모델 훈련하는 동안 모델의 성능을 손실값으로 측정. 손실값이 작을 수록 좋음.
- `metrics`
    - 훈련과 테스트 과정을 모니터링 할 때 사용되는 한 개 이상의 평가 지표<font size='2'>metric</font>를 포함하는 리스트로 지정.
    - 손실 함수값, 정확도 등 모델의 종류에 따라 다양한 평가 지표를 사용할 수 있음.
    - 분류 모델의 경우 일반적으로 정확도<font size='2'>accuracy</font>를 평가지표로 포함시킴.

### 데이터 전처리

머신러닝 모델에 따라 입력값이 적절한 형식을 갖춰야 한다.
앞서 두 개의 `Dense` 층과 `Sequential` 클래스로 지정된 모델의
입력값은 1차원 어레이 형식을 갖춰야 한다.

그런데 MNIST 데이터 샘플의 경우 
0부터 255 사이의 8비트 정수(`uint8`)로 이루어진 `(28, 28)` 모양의 2차원 어레이로 표현되었다.
이를 1차원 어레이로 변환하기 위해 `(28*28, )` 모양의 1차원 어레이로 변환한다.
또한 어레이의 각 항목을 0부터 1 사이의 32비트 부동소수점(`float32`)으로 변환한다.
이는 머신러닝 모델이 일반적으로 정수가 아닌 부동소수점 계산을 사용하기 때문이다.

```python
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255   # 0과 1사이의 값
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255     # 0과 1사이의 값
```

### 모델 훈련

모델 훈련은 컴파일된 모델의 `fit()` 메소드를 호출하면 된다.
MNIST 모델의 경우 지도 학습 모델이기에 입력 데이터셋과 타깃 데이터셋을 각각 첫째와 둘째 인자로 사용한다.

```python
model.fit(train_images, train_labels, epochs=5, batch_size=128)
```

- 첫째 인자: 훈련 데이터셋
- 둘째 인자: 훈련 라벨셋
- `epoths`: 에포크. 전체 훈련 세트 대상 반복 훈련 횟수.
- `batch_size`: 배치 크기. 배치 크기만큼의 훈련 데이터셋로 훈련할 때 마다 가중치 업데이트.

모델의 훈련 과정 동안 에포크가 끝날 때마다
평균 손실값과 평균 정확도를 계산하여 다음과 같이 출력한다.

```
Epoch 1/5
469/469 [==============================] - 5s 4ms/step - loss: 0.2551 - accuracy: 0.9263
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1044 - accuracy: 0.9693
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0683 - accuracy: 0.9793
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0504 - accuracy: 0.9847
Epoch 5/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0378 - accuracy: 0.9885
```

:::{admonition} 배치 크기와 스텝
:class: note

**스텝**<font size='2'>step</font>은 하나의 배치(묶음)에 대해 훈련하는 과정을 가리킨다.
위 코드에서 배치 크기(`batch_size`)가 128이기에 총 6만개의 훈련 샘플을 128개씩 묶는다.
따라서 469(60,000/128 = 468.75)개의 배치가 생성되며,
하나의 에포크 동안 총 469번의 스텝이 실행된다.
스텝이 끝날 때마다 사용된 배치 묶음에 대한 손실값과 정확도가 계산되어
에포크 단위로 평균값이 훈련 과정중에 보여지게 된다.
위 훈련은 총 5번의 훈련 에포크가 진행되며 최종적으로 훈련셋에 대한 정확도는 98.85%로 계산되었다.
:::

**모델 예측값 계산 과정**

전처리된 데이터가 신경망 모델에 전달되어 출력값으로 변환되는 과정을 묘사하면 다음과 같다.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist_2layers_arch.png?raw=true" style="width:600px;"></div>

- 손글씨 데이터 샘플 입력
    - 위 사진에서는 8을 가리키는 사진 샘플이 입력값으로 사용됨.
    - 784 개의 픽셀값으로 구성된 1차원 어레이로 변환
- 첫째 `Dense` 층
    - 입력된 784개의 픽셀값을 이용하여 512개의 값 생성. 
    - `relu()` 활성화 함수로 인해 음수는 모두 0으로 처리됨.
- 둘째 `Dense` 층
    - 첫째 `Dense` 층에서 생성된 512개의 값을 이용하여 10개의 값 생성.
    - `softmax()` 활성화 함수로 인해 모두 0과 1사이의 값으로 변환됨. 
        모든 값의 합이 1이 되며, 각각의 범주에 속할 확률을 가리킴.

**가중치 행렬과 출력값 계산**

아래 그림은 3개의 유닛으로 구성된 두 개의 `Dense` 층 사이에서 이뤄지는 데이터 변환을 한눈에 보여준다.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist01.png?raw=true" style="width:500px;"></div>

하나의 샘플에 대한 데이터 변환은 다음과 같이 계산된다.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist02.png?raw=true" style="width:500px;"></div>

행렬 연산으로 표현하면 다음과 같다.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist03a.png?raw=true" style="width:500px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://datascienceplus.com/mnist-for-machine-learning-beginners-with-softmax-regression/">MNIST For Machine Learning Beginners With Softmax Regression</a>&gt;</div></p>

:::{admonition} 선형 회귀 모델의 예측값 계산
:class: info

아래 식은 선형회귀 모델이 가중치 벡터와 편향을 이용하여 예측값을 계산하는 방식을 보여주는데,
신경망 모델에 포함된 하나의 유닛에 저장될 하나의 값을 계산하는 것과 동일하다.
단, 신경망 모델에 사용되는 사용되는 유닛마다 다른 가중치 벡터와 편향을 학습해야 한다는 점이 다르다.

$$
\hat y = w_1 \cdot x_1 + \cdots + w_n \cdot x_n + b
$$
:::

**스텝과 모델 훈련**

위 그림에서는 하나의 입력 샘플이 변환되는 과정을 보여준다.
하지만 모델의 훈련은 배치 단위로 이루어지며
배치에 포함된 모든 샘플에 대해 동시에 모델의 예측값을 계산한다.

예를 들어, 배치 크기가 128이면, 위 머신러닝 모델은 `128x784` 모양의 2차원 어레이를 입력값으로 사용하며,
각각의 층에서 이뤄지는 데이터 변환은 행렬 연산으로 계산된다.
활성화 함수는 행렬 연산으로 계산된 어레이를 입력값으로 사용하여
다음 층으로 전해줄 동일한 모양의 어레이를 계산한다.

하나의 스텝 과정에서 사용되는 가중치는 신경망 모델의 가중치 행렬은 훈련이 시작될 때 무작위로 초기화된다.
그리고 스텝을 반복하면서 타깃에 보다 가까운 예측값을 계산하는 가중치 행렬을 경사하강법을 활용하여 학습한다.
이 과정을 옵티마이저가 처리하며, 이 과정을 지정된 에포크만큼 반복하는 과정이 모델 훈련이다.

### 모델 활용

훈련된 모델을 이용하여 훈련에 사용되지 않은 손글씨 숫자 사진 10장에 대한 예측값을
`predict()` 메서드로 확인하는 방식은 다음과 같다.
`predict()` 메서드의 입력값이 하나의 샘플이 아닌 여러 개의 샘플에 대해 동시에 계산될 수 있음에 주의한다.
즉, 하나의 샘플에 대한 예측값을 계산하고자 하더라도 무조건 하나의 샘플 데이터로 구성된 2차원 어레이가 입력값으로 사용된다.

```python
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
```

출력값으로 각 사진에 대한 예측값으로 구성된 2차원 어레이가 계산된다.
각 항목은 각 입력값으로 사용된 손글씨 사진이 각 범주에 속할 확률을 갖는 
길이가 10인 1차원 어레이가 된다.
예를 들어 첫째 사진에 대한 예측값은 다음과 같다.

```python
>>> predictions[0]
array([5.6115879e-10, 6.5201892e-11, 3.8620074e-06, 2.0421362e-04,
       2.3715735e-13, 1.0822280e-08, 3.6126845e-15, 9.9979085e-01,
       2.0998414e-08, 1.0214288e-06], dtype=float32)
>>> predictions[0].argmax() 
7 
>>> predictions[0][7] 
0.99999106
```

위 예측값의 7번 인덱스의 값이 0.998 정도로 가장 높으며, 이는
0번 사진 입력 샘플이 숫자 7을 담고 있을 확률이 거의 100% 라고 예측했음을 의미한다.
실제로도 0번 사진은 숫자 7을 담고 있어서 이 경우는 정확하게 예측되었다.

```python
>>> test_labels[0] 
7
```

### 훈련된 모델 성능 평가

훈련에 사용되지 않은 테스트셋 전체에 대한 성능 평가를 위해 
`evaluate()` 메서드를 테스트셋과 테스트셋의 라벨셋을 인자로 해서 호출한다.

```python
>>> test_loss, test_acc = model.evaluate(test_images, test_labels)
313/313 [==============================] - 1s 3ms/step - loss: 0.0635 - accuracy: 0.9811
>>> print(f"test_acc: {test_acc}")
test_acc: 0.9811000227928162
```

`evaluate()` 메서드의 반환값 계산은 훈련 과정과 동일하게 배치 단위로 손실값과 앞서 모델을 컴파일할 때 지정한 정확도를
계산한 다음에 최종적으로 손실값과 정확도의 평균값을 반환한다.
배치 크기는 32가 기본값으로 사용되기에 총 313(10,000/32=312.5)번의 스텝이 진행되었다.

테스트 세트에 대한 정확도는 98.11% 이며 훈련 세트에 대한 정확도인 98.85% 보다 조금 낮다.
이는 모델이 훈련 세트에 대해 약간의 **과대 적합**<font size='2'>overfitting</font>이 발생했음을 의미한다. 
과대적합과 과대적합을 해결하는 다양한 방법에 대해서는 나중에 보다 자세히 다룬다.

## 텐서

MNIST 손글씨 데이터 분류 모델의 훈련에 사용된 훈련셋과 테스트셋은
넘파이 어레이, 즉 `numpy.ndarray`(이하 `np.ndarray`) 자료형으로 저장된다.
머신러닝에 사용되는 데이터셋은 일반적으로 넘파이 어레이와 같은 
**텐서**<font size='2'>tensor</font>에 저장된다.

텐서<font size='2'>tensor</font>는 데이터를 담은 모음 자료형을 가리키며
넘파이 어레이가 대표적인 텐서이다. 
텐서플로우 라이브러리는 자체의 `Tensor` 자료형인 `tensorflow.Tensor`(이하 `tf.Tensor`)를 제공한다.
`tf.Tensor`는 넘파이 어레이와 매우 유사하지만 GPU를 활용한 연산을 지원한다는 점에서 넘파이 어레이와 다르다.

앞서 신경망 모델을 구성할 때처럼 
여기서는 텐서플로우의 케라스 패키지인 `tensorflow.keras`를 기본 패키지로 사용하는데
케라스 신경망 모델의 입력, 출력값으로 넘파이 어레이를 기본으로 사용한다.
따라서 특별한 경우가 아니라면 넘파이 어레이를 데이터셋을 다룰 때 사용하면 된다.

### 텐서의 차원

텐서의 **차원**은 텐서의 표현에 사용된 **축**<font size='2'>axis</font>의 수로 
결정되며 **랭크**<font size='2'>rank</font>라 불리기도 한다.

- 0차원(0D) 텐서 (랭크-0 텐서)
    - 정수 한 개, 부동소수점 한 개 등 하나의 수를 표현하는 텐서. 
    - **스칼라**<font size='2'>scalar</font>라고도 불림.
        ```
        np.array(12)
        np.array(1.34)
        ```

- 1차원(1D) 텐서 (랭크-1 텐서)
    - 수로 이루어진 리스트 형식의 텐서. 
    - **벡터**<font size='2'>vector</font>로 불리며 한 개의 축을 가짐.
        ```
        np.array([12, 3, 6, 14, 7])
        ```

- 2차원(2D) 텐서 (랭크-2 텐서)
    - 행<font size='2'>row</font>과 열<font size='2'>column</font> 두 개의 축을 가짐. 
    - **행렬**<font size='2'>matrix</font>로도 불림.
        ```
        np.array([[5, 78, 2, 34, 0],
                [6, 79, 3, 35, 1],
                [7, 80, 4, 36, 2]])
        ```
    - 흑백 사진 데이터를 2D 텐서로 표현 가능. 아래 그림 참고.
        <br>
        <div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist05.png?raw=true" style="width:600px;"></div>

        <p><div style="text-align: center">&lt;그림 출처: <a href="https://towardsdatascience.com/exploring-how-neural-networks-work-and-making-them-interactive-ed67adbf9283">Towards data science: Mikkel Duif(2019)</a>&gt;</div></p>

- 3차원(3D) 텐서 (랭크-3 텐서)
    - 행, 열, 깊이 세 개의 축 사용.
    - 동일 모양의 2D 텐서로 구성된 벡터로 이해 가능. 
        예를 들어 흑백 사진 샘프로 구성된 데이터셋이 3D 텐서로 표현됨.
        ```
        np.array([[[5, 78, 2, 34, 0],
                    [6, 79, 3, 35, 1],
                    [7, 80, 4, 36, 2]],
                    [[5, 78, 2, 34, 0],
                    [6, 79, 3, 35, 1],
                    [7, 80, 4, 36, 2]],
                    [[5, 78, 2, 34, 0],
                    [6, 79, 3, 35, 1],
                    [7, 80, 4, 36, 2]]])
        ```
    - 또는 동일한 길이의 벡터를 항목으로 사용하는 2D 텐서로 이해 가능.
        예를 들어, RGB로 구성된 사진 데이터가 3D 텐서로 표현됨. 아래 그림 참고. 
        <br>
        <div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-rgb-3d-1.png?raw=true" style="width:600px;"></div>
        <div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-rgb-3d-2.png?raw=true" style="width:600px;"></div>

        <p><div style="text-align: center">&lt;그림 출처: <a href="https://dev.to/sandeepbalachandran/machine-learning-going-furthur-with-cnn-part-2-41km">Machine Learning - Going Furthur with CNN Part 2</a>&gt;</div></p>



- 4D 텐서 (랭크-4 텐서)
    - 3D 텐서로 이루어진 벡터
    - 예를 들어 컬러 사진 데이터로 구성된 훈련셋, 연속된 사진으로 처리될 수 있는 한 편의 동영상 등이 4D 텐서로 표현됨.

- 5D 텐서 (랭크-5 텐서)
    - 4D 텐서로 이루어진 벡터
    - 예를 들어 동영상 데이터로 구성된 훈련셋 등이 5D 텐서로 표현됨.

6D 텐서는 5D 텐서로 이루어진 벡터, 7D 텐서는 6D 텐서로 구성된 벡터 등으로 임의의 차원의 텐서를 정의할 수 있지만
일반적이지는 않다.

:::{admonition} 벡터의 차원
:class: caution

**벡터의 길이**를 **차원**이라 부르기도 한다.
예를 들어, `np.array([12, 3, 6, 14, 7])`는 5차원 벡터다.
따라서 벡터의 차원인지, 텐서의 차원인지 명확히 구분할 필요가 있다.
:::

### 텐서 주요 속성

텐서의 주요 속성 세 가지는 다음과 같으며, 넘파이 어레이의 경우와 동일하다.

- `ndim` 속성: 텐서의 차원 저장. 
    예를 들어 MNIST 훈련셋 어레이의 차원은 3.
    ```python
    >>> train_images.ndim 
    3
    ```

- `shape` 속성: 텐서의 모양을 튜플로 저장. 
    각 항목은 축(axis)별로 사용되는 벡터의 크기를 가리킴.
    예를 들어 MNIST의 훈련셋은 3개의 축으로 구성됨.
    0번 축은 6만개의 샘플 데이터를,
    1번 축은 각 사진에 사용된 28개의 세로 픽셀 데이터를,
    2번 축은 각 사진에 사용된 28개의 가로 픽셀 데이터를 가리킴.
    ```python
    >>> train_images.shape
    (60000, 28, 28)
    ```

- `dtype` 속성: 텐서에 포함된 항목의 통일된 자료형.
    `float16`, `float32`,`float64`, `int8`, `uint8`, `string` 등이 
    가장 많이 사용됨.
    예를 들어, MNIST 훈련셋에 포함된 사진의 픽셀 정보는 0과 255 사이의
    정수로 표현되며 따라서 `unit8` 자료형을 사용함.
    ```python
    >>> train_images.dtype
    uint8
    ```

데이터셋의 차원과 모양 정보와 인덱싱, 슬라이싱 기능을 이용하여
샘플을 확인하고 배치를 생성하는 일 등을 처리할 수 있다.

**인덱싱**

예를 들어, 훈련셋에 포함된 4번 인덱스의 사진, 즉 훈련셋의 5번째 사진을 다음처럼 선택하여 확인할 수 있다.
```python
>>> import matplotlib.pyplot as plt
>>> digit = train_images[4]
>>> plt.imshow(digit, cmap=plt.cm.binary)
>>> plt.show()
```

<img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch02-mnist4.png?raw=true" style="width:250px;">
<br>

위 사진은 숫자 9를 가리키는 것으로 보인다.
실제 해당 샘플의 라벨이 9로 확인된다.

```python
>>> train_labels[4] 
9
```

<br>

**슬라이싱**

케라스를 포함하여 대부분의 딥러닝 모델은 훈련 세트 전체를 한꺼번에 처리하지 않고
지정된 크기(`batch_size`)의 배치를 이용하여 스텝 단위로 훈련한다.
앞서 살펴본 모델의 배치 크기는 128이었다.
크기가 128인 첫째 배치는 다음과 같이 지정한다.

```python
>>> batch = train_images[:128]
```

다음은 둘째 배치를 지정한다.

```python
>>> batch = train_images[128: 256]
```

훈련셋의 `n`번째 배치는 다음과 같이 지정할 수 있다.

```python
>>> batch = train_images[128 * n:128 * (n + 1)]
```

### 텐서 실전 예제

**(1) 2D 텐서 실전 예제**

각각의 샘플이 지정된 개수의 특성으로 구성된 벡터로 표현된다.
전체 데이터셋은 `(샘플 수, 특성 수)` 모양의 2D 텐서로 표현된다.

- 예제 1: [캘리포니아 구역별 인구조사 데이터셋](https://codingalzi.github.io/handson-ml3/end2end_ml_project.html)
    - 샘플: 10개의 특성 사용. 따라서 `(10,)` 모양의 벡터로 표현됨.
    - 데이터셋: 20,640개의 구역별 데이터 포함. 따라서 `(20640, 10)` 모양의 2D 텐서로 표현 가능.
        <br>
        <img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch02/housing-data.png" style="width:600px;">

- 예제 2
    - 샘플: 문장에 사용된 단어들의 빈도를 모아놓은 벡터. 
        예를 들어, 지정된 2만 개의 단어 각각이 지정된 문장에 사용된 빈도를 측정하여
        `(20000,)` 모양의 벡터로 표현 가능.
    - 데이터셋: 10만 개의 문장을 대상으로 2만 개 단어의 사용빈도를 측정한 데이터셋은 
        `(100000, 20000)` 모양의 2D 텐서로 표현 가능.

- 예제 3: 사이킷런 모델의 입력 데이터셋은 기본적으로 2D 텐서를 사용함.

**(2) 3D 텐서 실전 예제**

증시 데이터 등의 시계열 데이터와 트위터 데이터 등의 순차 데이터를 다룰 때 사용하며
`(샘플 수, 타임 스텝 수, 특성 수)` 모양의 3D 텐서로 표현된다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/ch02-timeseries_data.png" style="width:350px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

- 예제 1
    - 샘플: 1분마다 하루 총 390번 (현재 증시가, 지난 1분 동안 최고가, 지난 1분 동안 최저가)를 
        측정한 데이터. `(390, 3)` 모양의 2D 텐서로 표현.        
    - 데이터셋: 250일 동안 측정한 데이터셋은 `(250, 390, 3)` 모양의 3D 텐서로 표현.

- 예제 2
    - 샘플: 하나의 트위터 데이터(트윗)는 최대 280개의 문자로 구성되고, 사용할 수 있는 문자가 총 128 개일 때
        트위터 샘플 하나를 `(280, 128)` 모양의 2D 텐서로 표현 가능함. 
        각각의 항목은 128개의 문자 각각의 사용여부를 확인해주는 0 또는 1.
    - 데이터셋: 백만 개의 샘플로 구성된 트위터 데이터셋은 `(1000000, 280, 128)` 모양의 3D 텐서로 표현 가능.

- 예제 3: 흑백 사진으로 구성된 데이터셋
    - 샘플: `28x28` 크기의 (흑백) 손글씨 사진.
        `(28, 28)` 모양의 2D 텐서로 표현 가능.
    - MNIST 훈련 데이터셋: 총 6만개의 (흑백) 손글씨 사진으로 구성됨.
        `(60000, 28, 28)` 모양의 3D 텐서로 표현 가능.

**(3) 4D 텐서 실전 활용 예제**

한 장의 컬러 사진 샘플은 일반적으로 
`(높이, 너비, 채널 수)` 또는 `(채널 수, 높이, 너비)`
모양의 3D 텐서로 표현한다. 
따라서 컬러 사진으로 구성된 데이터셋은 
`(샘플 수, 높이, 너비, 채널 수)` 또는 `(샘플 수, 채널 수, 높이, 너비)`
모양의 4D 텐서로 표현된다.

RGB를 사용하는 컬러 어미지는 3개의 커널을, 흑백 사진은 1개의 채널을 갖는다. 
예를 들어 `256x256` 크기의 컬러 사진 128개를 갖는 데이터셋은
`(128, 256, 256, 3)` 모양 4D 텐서로 표현된다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/ch02-image_data.png" style="width:350px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

반면에 `28x28` 크기의 흑백 사진 128개를 갖는 데이터셋 또는 배치는
`(128, 28, 28, 1)` 모양 4D 텐서로 표현된다.
하지만 MNIST의 경우처럼 흑백 사진 데이터셋은 `(128, 28, 28)` 모양의 3D로 표현하기도 한다.
예를 들어 `(3, 3, 1)` 모양의 3D 텐서를 `(3, 3)` 모양의 텐서로 표현할 수 있다.

```python
>>> tensor331 = np.array([[[1], [2], [3]],
                          [[4], [5], [6]],
                          [[7], [8], [9]]])
>>> tensor331.reshape(3, 3)
np.array([[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]])
```

**(4) 5D 텐서 실전 예제**

동영상은 프레임<font size='2'>frame</font>으로 구성된 순차 데이터다.
프레임은 한 장의 컬러 사진이며, 
`(높이, 너비, 채널 수)` 모양의 3D 텐서로 표현된다.
따라서 하나의 동영상은 `(프레임 수, 높이, 너비, 채널 수)` 모양의 4D 텐서로
표현된다.
이제 여러 개의 동영상으로 이루어진 데이터셋은 
`(동영상 수, 프레임 수, 높이, 너비, 채널 수)` 모양의 5D 텐서로 표현된다.

예를 들어, `144x256` 크기의 프레임으로 구성된 60초 동영상이 초당 4개의 프레임을 사용한다면
동영상 한 편은 `(240, 144, 256, 3)` 모양의 4D 텐서로 표현된다.
따라서 동영상 10 편으로 구성된 데이터셋은 `(10, 240, 144, 256, 3)` 모양의 5D 텐서로 표현된다.

## 텐서 연산

### 신경망 모델의 주요 연산

신경망 모델의 훈련은 기본적으로 텐서와 관련된 몇 가지 연산으로 이루어진다. 
예를 들어 이전 신경망에 사용된 층을 살펴보자.

```python
keras.layers.Dense(512, activation="relu")
keras.layers.Dense(10, activation="softmax")
```

첫째 층이 하는 일은 데이터 변환이며 실제로 이루어지는 연산은 다음과 같다.

- `W1`: 첫째 층에서 학습되는 가중치 행렬
- `b1`: 첫째 층에서 학습되는 편향 벡터

`output1 = relu(np.dot(input, W1) + b1)`

사용된 세부 연산은 다음과 같다. 

- 텐서 점곱: `np.dot()` 함수에 의해 행렬 곲 계산
- 텐서 덧셈: `+`. 텐서 대상 항목별 덧셈.
- 활성화 함수: `relu()` 함수. 음수 항목을 모두 0으로 대체함.

둘째 층은 다른 가중치 행렬과 편향을 학습하여 데이터를 변환한다.

- `W2`: 둘째 층에서 학습되는 가중치 행렬
- `b2`: 둘째 층에서 학습되는 편향 벡터

`output2 = softmax(np.dot(input, W2) + b2)`

`softmax()` 함수는 분류 신경망 모델의 마지막 층인 **출력층**에서 사용되는 활성화 함수이며
입력 샘플이 클래스별로 해당 클래스에 속할 확률을 계산한다. 
위 모델의 경우 10개 유닛에서 계산된 값들을 이용하여 10개 각 클래스별로 속할
확률을 계산하며, 확률값의 합은 1이 되도록 한다.

**항목별 연산과 브로드캐스팅**

앞서 언급된 연산과 함수 중에서 덧셈 연산은 텐서에 포함된 항목별로 연산이 이뤄진다.
아래 그림은 텐서의 항목별 덧셈과 브로드캐스팅이 작동하는 방식을 보여준다.

<div align="center"><img src="https://scipy-lectures.org/_images/numpy_broadcasting.png" style="width:750px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://scipy-lectures.org/intro/numpy/operations.html">Scipy Lecture Notes</a>&gt;</div></p>

텐서 연산과 브로드캐스팅을 가능한 모든 경우에 적용된다.
아래 그림은 3차원 텐서와 2차원 텐서의 연산에 브로드캐스팅이 
자동으로 적용되는 과정을 보여준다.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/pydata/master/notebooks/images/broadcasting12.png" style="width:300px;"></div>

**유니버설 함수**

덧셈, 뺄셈, 곱셈, 나눗셈의 사칙 연산 이외에 다른 많은 연산과 함수도 항목별로 적용된다.
예를 들어, `relu()` 함수가 0보다 작은 항목을 0으로 대체하는 데에 사용하는 `np.maximum()` 함수가
텐서의 항목 각각에 대해 작동하는 과정을 보여준다.
이와 같이 항목별로 작동하는 함수를 **유니버설**<font size='2'>universal</font> 함수라 부른다.

```
relu(t) = np.maximum(t, 0)
```

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch01-universal_functions01.jpg" style="width:500px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.sharpsightlabs.com/blog/numpy-maximum/">Sharp Sight - How to Use the Numpy Maximum Function</a>&gt;</div></p>

:::{admonition} `softmax()` 함수
:class: warning

유니버설 함수만 활성화 함수로 사용되는 것은 아니다. 
`softmax()` 함수는 유니버설 함수가 아니며, 각 유닛에서 계산된 값들의 상대적 크기를 계산한다.
:::

**텐서 점곱**

**텐서 점곱**<font size='2'>tensor dot product</font> 함수는
두 벡터의 내적 또는 두 행렬의 곱을 계산할 때 사용된다.
아래 그림에서 보여지는 것처럼 두 인자의 유형에 따라 다르게 작동한다.

- 1D와 스칼라의 점곱: 항목 별 배수 곱셈
- 1D와 1D의 점곱: 벡터 내적
- 1D와 2D의 점곱: 행렬 곱셈
- 2D와 2D의 점곱: 행렬 곱셈

<div align="center"><img src="https://blog.finxter.com/wp-content/uploads/2021/01/numpy_dot-1-scaled.jpg" style="width:500px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://blog.finxter.com/dot-product-numpy/">finxter - NumPy Dot Product</a>&gt;</div></p>

:::{admonition} 텐서의 곱셈(`*`)
:class: warning

텐서 점곱(`np.dot()`)과 텐서 곱셈(`*`)은 다르다.
텐서 곱셈은 앞서 브로드캐스팅과 연관지어 설명된 것처럼 동일 모양의 두 텐서를
항목별로 곱해서 동일 모양의 새로운 텐서를 생성한다.

```python
>>> a = np.array([1.0, 2.0, 3.0])
>>> b = np.array([2.0, 2.0, 2.0])
>>> a * b
array([2.,  4.,  6.])

>>> a = np.array([1.0, 2.0, 3.0])
>>> b = 2.0
>>> a * b
array([2.,  4.,  6.])

>>> a = np.array([[ 0.0,  0.0,  0.0],
                  [10.0, 10.0, 10.0],
                  [20.0, 20.0, 20.0],
                  [30.0, 30.0, 30.0]])
>>> b = np.array([1.0, 2.0, 3.0])
>>> a*b
array([[ 0.,  0.,  0.],
       [10., 20., 30.],
       [20., 40., 60.],
       [30., 60., 90.]])
```
:::

**텐서 모양 변형**

머신러닝 모델은 입력 텐서의 모양을 제한한다. 
앞서 사용한 MNIST 데이터셋 분류 모델은 `Dense` 층으로 구성되었는데, 
입력값으로 사용되는 데이터셋은 2차원 텐서로 표현되어 있어야 한다.

```python
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

반면에 `tensorflow.keras.datasets`에서 불러온 
`train_images`와 `test_images` 는 각각
`(60000, 28, 28)`와 `(10000, 28, 28)` 모양의 3차원 텐서다.

```python
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
```

따라서 모델을 훈련 및 테스트하고 실전에 활용할 때는 입력값을 항상
적절한 모양의 2차원 텐서로 변형해야 한다.
이를 위해 다음과 같이 `reshape()` 텐서 메서드를 활용한다.
아래 코드는 `(60000, 28, 28)` 모양의 훈련셋인 3차원 텐서를 동일 개수의 항목을 갖는
`(60000, 784)` 모양의 2차원 텐서로 변형한다.

```python
train_images = train_images.reshape((60000, 28 * 28))
```

:::{admonition} 넘파이 어레이 연산
:class: info

텐서 연산의 기본이 되는 넘파이 어레이 연산, 유니버설 함수, 텐서 모양 변형 등에 대한
보다 자세한 설명은 
[파이썬 데이터 분석](https://codingalzi.github.io/datapy/intro.html)을 참고한다.
:::

### 텐서 연산의 기하학적 의미

신경망 모델에 사용되는 연산과 함수들의 기능을 
기하학적으로 설명할 수 있다.

- 이동: 벡터 합

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/translation.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

- 회전: 점 곱

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/rotation.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

- 스케일링: 점 곱

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/scaling.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

- 아핀 변환

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/affine_transform.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

- 아핀 변환과 relu 활성화 함수

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/dense_transform.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

**신경망 모델 연산의 의미**

신경망은 기본적으로 앞서 언급된 텐서 연산의 조합을 통해
고차원 공간에서의 매우 복잡한 기하학적 변환을 수행한다.

예를 들어, 빨간 종이와 파란 종이 두 장을 겹친 뭉개서 만든 종이 뭉치를
조심스럽게 조금씩 펴서 결국 두 개의 종이로 구분하는 것처럼
신경망 모델은 뒤 섞인 두 개 클래스로 구성된 입력 데이터셋을
여러 층을 통해 변환하면서 결국엔 두 개의 데이터셋으로 구분하는
방법을 알아낸다. 

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/ch02-geometric_interpretation_4.png" style="width:400px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

## 연습문제

1. [(실습) 신경망 구성 요소](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-building_blocks_of_NN.ipynb)